## Webscraping notebook

In [79]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [80]:
#loading downloaded data
df=pd.read_json('~/Downloads/reviews_Electronics_5.json', lines=True)

In [81]:
#examining text of reviews
df.reviewText

0          We got this GPS for my husband who is an (OTR)...
1          I'm a professional OTR truck driver, and I bou...
2          Well, what can I say.  I've had this unit in m...
3          Not going to write a long review, even thought...
4          I've had mine for a year and here's what we go...
                                 ...                        
1689183    Burned these in before listening to them for a...
1689184    Some people like DJ style headphones or earbud...
1689185    I&#8217;m a big fan of the Brainwavz S1 (actua...
1689186    I've used theBrainwavz S1 In Ear Headphones, a...
1689187    Normally when I receive a review sample I can ...
Name: reviewText, Length: 1689188, dtype: object

In [4]:
#error asins
# df= df[df.asin!='B00004SYNX']
# df= df[df.asin!='B00000J1SC']
# df= df[df.asin!='B000067RVL']
# df= df[df.asin!='B000068O53']
# df= df[df.asin!='B00006B8EC']
# df= df[df.asin!='']

### The links, links1, and links2 pkl files contain scraped links of the Amazon products. They are error free.

In [ ]:
url=

In [57]:
with open ('links.pickle', 'wb') as f:
    pickle.dump(links, f, 5)

In [67]:
with open ('links1.pickle', 'wb') as f:
    pickle.dump(links1, f, 5)

In [75]:
with open ('links2.pickle', 'wb') as f:
    pickle.dump(links2, f, 5)

### Webscraping the item name, item tags, and price to be added to df

In [125]:
#getting our urls ready for webscraping
df1= pd.Series(links)
df2=pd.Series(links1)
df3=pd.Series(links2)
frames=[df1,df2,df3]
url_series=pd.concat(frames, axis=0)
len(url_series)

26885

### Scraping the item name, item tag, and price of the item

In [349]:
from tqdm import tqdm
import time
itemz=[]
linkz_p=[]
tagz_p=[]
item_namezz=[]
driver = webdriver.Chrome()
for x in tqdm(url_series):
    try:
        driver.get(str(x))
        content=driver.find_elements_by_class_name('a-price')
        content1=driver.find_elements_by_class_name('a-link-normal.a-color-tertiary')
        content2=driver.find_elements_by_id('productTitle')
        see= [x.text for x in content]
        item= [x.text for x in content1]
        item_namez= [x.text for x in content2]
        itemz.append(item)
        tagz_p.append(see)
        item_namezz.append(item_namez)
        linkz_p.append(x)
    except:
        continue
        time.sleep(.0005)
        driver.close()

  0%|                                                 | 0/26885 [00:00<?, ?it/s]/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_1871/3200647733.py:11: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  content=driver.find_elements_by_class_name('a-price')
/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_1871/3200647733.py:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  content1=driver.find_elements_by_class_name('a-link-normal.a-color-tertiary')
/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_1871/3200647733.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  content2=driver.find_elements_by_id('productTitle')
100%|██████████████████████████████████| 26885/26885 [27:17:48<00:00,  3.66s/it]


### Cleaning and preparing scraped data for pandas

In [352]:
#cleaning prices
prices=[]
for x in tagz_p:
    if len(x) > 1:
        prices.append(x[0])
    else:
        prices.append(x)


In [353]:
cleaned_prices=[str(x).replace('\n', '.') for x in prices]

In [354]:
cleaned_prices=[str(x).replace('[', '') for x in cleaned_prices]

In [355]:
cleaned_prices=[str(x).replace(']', '') for x in cleaned_prices]

In [356]:
cleaned_prices=[str(x).replace("'", '') for x in cleaned_prices]

In [357]:
cleaned_prices=pd.Series(cleaned_prices)

In [358]:
#cleaning tags
items=pd.Series(itemz)
test=[items.iloc[x] for x in range(len(items))]


In [359]:
tags=[]
for i in range(len(test)):
    if len(test[i]) >=1:
        tags.append(test[i][-1])
    else:
        tags.append(np.nan)

In [360]:
tags=pd.Series(tags)

In [361]:
#cleaning links to join on asin
links=[str(x)[-10:] for x in linkz_p]

In [362]:
links=pd.Series(links)

['Pyrus Electronics 8gb Touch Multimedia Player with 4.3 Inch Touch Screen and Built in Wireless in Car Fm Transmitter']

In [377]:
df_scrape=pd.concat([item_namezz,links, tags, cleaned_prices], axis=1)
df_scrape=pd.DataFrame(df_scrape)

In [380]:
df_scrape.head()

,0,1,2,3
0,[Rand McNally 528881469 7-inch Intelliroute TN...,0528881469,Trucking GPS,$399.99
1,[Barnes & Noble HDTV Adapter Kit for NOOK HD a...,0594451647,Chargers & Adapters,$29.99
2,[Barnes & Noble OV/HB Universal Power Kit for ...,0594481813,Power Adapters,$16.99
3,"[VideoSecu 24"" Long Arm TV Wall Mount Low Prof...",0972683275,TV Ceiling & Wall Mounts,$34.99
4,"[Barnes & Noble NOOK Tablet 16gb (Color, BNTV2...",1400501466,Tablets,$49.99


In [381]:
df_scrape.rename(columns={ 0: 'item', 1:'asin', 2:'tag', 3:'price'}, inplace=True)

In [382]:
df_scrape

,item,asin,tag,price
0,[Rand McNally 528881469 7-inch Intelliroute TN...,0528881469,Trucking GPS,$399.99
1,[Barnes & Noble HDTV Adapter Kit for NOOK HD a...,0594451647,Chargers & Adapters,$29.99
2,[Barnes & Noble OV/HB Universal Power Kit for ...,0594481813,Power Adapters,$16.99
3,"[VideoSecu 24"" Long Arm TV Wall Mount Low Prof...",0972683275,TV Ceiling & Wall Mounts,$34.99
4,"[Barnes & Noble NOOK Tablet 16gb (Color, BNTV2...",1400501466,Tablets,$49.99
...,...,...,...,...
26779,[GSI Super Quality Motorcycle Helmet Dual-Side...,B003JUN9YW,Over-Ear Headphones,$32.99
26780,[Nook Barnes and Nobles Electronic eBook Reade...,B003JUXWSU,Covers,
26781,[Menotek Mini DisplayPort to HDMI Converter ca...,B003JV3XZ6,HDMI Cables,$10.29
26782,[Durable Protective Digital Camera Bag with Le...,B003JVTOA4,Camera Cases,$22.54


In [383]:
load_df=df.merge(df_scrape, how='inner', on='asin')

In [384]:
#pickle file to be used in EDA_NB
import pickle
with open ('load_df.pickle', 'wb') as f:
    pickle.dump(load_df, f, 5)

In [385]:
load_df.shape

(702972, 12)